In [2]:
# working dir
import os
import sys

cwd = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(cwd))
sys.path.append(root_dir)

print(root_dir)

c:\Users\david\Desktop\Main\03_Uni\WWI21DSA\02_Vorlesungen\06_Projektrealisierung\Projektrealisierung


In [3]:
# Imports and settings
import pandas as pd
import random

random.seed(42)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [4]:
# Load data
abcd = pd.read_csv("../../src/data/ABCD_tripfiles_preprocessed.csv")
# mnop = pd.read_csv("../../src/data/MNOP_tripfiles_preprocessed.csv")
# zyxw = pd.read_csv("../../src/data/ZYXW_tripfiles_preprocessed.csv")
# datasets = {"abcd": abcd, "mnop": mnop, "zyxw": zyxw}

C:\Users\david\AppData\Local\Temp\ipykernel_8744\1085071673.py:2: DtypeWarning: Columns (0,13,14,15,16,17,18,19,20,21,22,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  abcd = pd.read_csv("../../src/data/ABCD_tripfiles_preprocessed.csv")


In [5]:
meta = ['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name']

message = ['header_line', 'entry_details']

header_cols = ['header_category', 'header_id']

extracted_cols = ['scheduleState',
       'departureAirport', 'departureTime', 'arrivalAirport', 'arrivalTime',
       'aircraftRegistration', 'aircraftSubtype', 'aircraftVersion',
       'serviceType', 'nonOperational', 'flifoCancel', 'sequenceNumber',
       'bestTimes', 'mvtActual']

weight_cols = ['START_WI_weight', 'START_WI_index',
       'DO_WI_weight', 'DO_WI_index', 'PAX_WI_weight', 'PAX_WI_index',
       'TOTAL_DEADLOAD_WI_weight', 'TOTAL_DEADLOAD_WI_index', 'TOTAL_LOAD_WI',
       'TOTAL_TRAFFIC_LOAD', 'FUEL_INDEX', 'AZFW', 'ATOW', 'ALAW', 'ATXW',
       'AFT_LIMIT_ZFW', 'FWD_LIMIT_ZFW', 'AFT_LIMIT_TOW', 'FWD_LIMIT_TOW',
       'AFT_LIMIT_LAW', 'FWD_LIMIT_LAW', 'LIZFW', 'LITOW', 'LILAW',
       'MAC_AT_ZFW', 'MAC_AT_TOW', 'MAC_AT_LAW', 'DEADLOAD_MAC', 'UNDERLOAD',
       'LIMITING_WEIGHT', 'ALLOWED_TOW', 'ALLOWED_ZFW', 'ALLOWED_LAW',
       'ALLOWED_TXW', 'STABTO', 'OPTIMAL_TRIM', 'IDEAL_ADDITIONAL_LOAD_AFT',
       'IDEAL_ADDITIONAL_LOAD_FWD', 'TAIL_TIPPING_WI_weight',
       'TAIL_TIPPING_WI_index', 'TAIL_TIPPING_INDEX_EXCEEDED',
       'FWD_MOVABLE_PAX', 'AFT_MOVABLE_PAX', 'INDEX_OUT_OF_BALANCE',
       'LOAD_TO_AFT', 'LOAD_TO_FWD', 'ESTIMATED_TRAFFIC_LOAD', 'ESTIMATED_ZFW',
       'DELTA_ZFW', 'ZFW_TOLERANCE_EXCEEDED']


In [6]:
# def create_flight_id(df):
#     df["flight_id"] = df["departureAirport"] + df["arrivalAirport"] + df["departureTime"] + df["arrivalTime"]

# create_flight_id(abcd)

Jetzt mal schauen wie man eine valid flight id hinbekommen kann

In [7]:
grouped = abcd.groupby("header_id")
grouped_count = grouped["action_name"].value_counts().reset_index()
print(grouped_count.columns)
grouped_count.head(10)

Index(['header_id', 'action_name', 'count'], dtype='object')


,header_id,action_name,count
0,10003807ae9efee7,CalculateWeightAndTrimAction,2
1,10003807ae9efee7,CheckinMsgProcessor,2
2,10003807ae9efee7,StorePaxDataAction,2
3,100187a20ab635a5,CalculateWeightAndTrimAction,2
4,100187a20ab635a5,CreateZFWMessageAction,2
5,100187a20ab635a5,UpdateEstimatesAction,2
6,1001b6bafe526ee0,CalculateWeightAndTrimAction,2
7,1001b6bafe526ee0,UpdateFuelDataAction,2
8,1001ba0d2120e5cb,CalculateWeightAndTrimAction,2
9,1001ba0d2120e5cb,CheckinMsgProcessor,2


In [8]:
grouped_count_filtered = grouped_count.query("action_name == 'CalculateWeightAndTrimAction'")
valid_header_ids = set(grouped_count_filtered["header_id"].unique())
invalid_header_ids = set(abcd["header_id"].unique()) - valid_header_ids

print(
    len(abcd["header_id"].unique()),
    len(invalid_header_ids),
    len(valid_header_ids),
)

121282 17351 103931


von 121282 header ids haben 103931 mind. 1x die action CalculateWeightAndTrimAction (aus der der Ankunftsflughafen und die abflugs- und ankunftszeiten hervorgehen)

In [1]:
# invalid_id_sample = random.sample(invalid_header_ids, 5)

# abcd[abcd["header_id"].isin(invalid_id_sample)].sort_values("header_id")

In [11]:
action_count_series = abcd.drop_duplicates(subset=['header_id', 'action_name'])['action_name'].value_counts().reset_index()
action_count_df = pd.DataFrame(action_count_series)
action_count_df["percent"] = action_count_df["count"] / abcd["header_id"].nunique()


action_count_df


,action_name,count,percent
0,CalculateWeightAndTrimAction,103931,0.856937
1,StorePaxDataAction,75727,0.624388
2,CheckinMsgProcessor,66891,0.551533
3,UpdateLoadTableAction,11586,0.095529
4,ASMMsgProcessor,11219,0.092503
5,UpdateCrewDataAction,6911,0.056983
6,AssignLCCAction,4787,0.039470
7,EstimateStorePaxDataAction,4754,0.039198
8,UpdateFlightAction,4734,0.039033
9,PAXBOOKINGINMsgProcessor,4439,0.036601


In [28]:
# for action in abcd["action_name"].unique():
#     # for dataset_name, dataset in datasets.items():
#     for cat in ['received', 'saved', 'sent']:
#         filtered_df = abcd[
#             (abcd["header_category"] == cat) &
#             (abcd["action_name"] == action)
#         ]
#         (f"header_category == {cat} & action_name == {action}")
#         print(action, cat)
#         display(filtered_df[message].head(1))

In [22]:

from IPython.display import display, HTML
# Function to display the DataFrame in a fixed-size window with scrollbars
def display_dataframe_with_scrollbars(df, width='1600px', height='800px'):
    style = f"""
    <style>
    .dataframe-container {{
        width: {width};
        height: {height};
        overflow: auto;
        position: relative;
    }}
    .dataframe-container table {{
        width: 100%;
        border-collapse: collapse;
    }}
    .dataframe-container thead th {{
        position: sticky;
        top: 0;
        background: white;
    }}
    </style>
    """
    html = f"""
    {style}
    <div class="dataframe-container">
    {df.to_html(index=False)}
    </div>
    """
    display(HTML(html))


In [27]:
# random_flight_number = random.choice(abcd["flight_number"].unique())

# df = abcd.loc[abcd["flight_number"] == random_flight_number, meta + message + header_cols].sort_values("creation_time")[meta + message + header_cols]
# # 
# display_dataframe_with_scrollbars(df)